In [1]:
numpad = {
   '7' : (0,0),
   '8' : (0,1),
    '9' : (0,2),
    '4' : (1,0),
    '5' : (1,1),
    '6' : (1,2),
    '1' : (2,0),
    '2' : (2,1),
    '3' : (2,2),
    '0' : (3,1),
    'A':(3,2)
    
}

numpad_coords = {v: k for k, v in numpad.items()}

In [2]:
keypad = {
    '^' : (0,1),
    '<' : (1,0),
    'v' : (1,1),
    '>' : (1,2),
    'A' : (0,2)
}

keypad_coords = {v: k for k, v in keypad.items()}

In [3]:
def addT(a,b):
    return (a[0]+b[0],a[1]+b[1])

In [4]:

def getPath(coords, target, pad_coords):
    paths = []
    start = (coords,'')
    ol, cl = [start], []

    while ol: 

        position, path = ol[0]
        #print(ol)
        cl.append(ol[0])
        ol.pop(0) 

        if position == target:
            path+='A'
            paths.append(path)


        directions = []

        if position[0]>target[0]:
            directions.append((-1,0))

        if position[1]>target[1]:
            directions.append((0,-1))

        if position[0]<target[0]:
            directions.append((1,0))

        if position[1]<target[1]:
            directions.append((0,1))

        for way in directions:

            if addT(position,way) in pad_coords:

                if way == (-1,0):
                    newpath = path + '^'
                if way == (1,0):
                    newpath = path + 'v'
                if way == (0,-1):
                    newpath = path + '<'
                if way == (0,1):
                    newpath = path + '>'

                ol.append((addT(position,way),newpath))

    return paths 

In [5]:
from functools import lru_cache

@lru_cache(maxsize=None) 
def getPathCache(coords, target):
    paths = []
    start = (coords,'')
    ol, cl = [start], []

    while ol: 

        position, path = ol[0]
        #print(ol)
        cl.append(ol[0])
        ol.pop(0) 

        if position == target:
            path+='A'
            paths.append(path)


        directions = []

        if position[0]>target[0]:
            directions.append((-1,0))

        if position[1]>target[1]:
            directions.append((0,-1))

        if position[0]<target[0]:
            directions.append((1,0))

        if position[1]<target[1]:
            directions.append((0,1))

        for way in directions:

            if addT(position,way) in keypad_coords:

                if way == (-1,0):
                    newpath = path + '^'
                if way == (1,0):
                    newpath = path + 'v'
                if way == (0,-1):
                    newpath = path + '<'
                if way == (0,1):
                    newpath = path + '>'

                ol.append((addT(position,way),newpath))

    return paths

In [6]:
codes = ['341A',
'480A',
'286A',
'579A',
'149A']

In [ ]:
from itertools import product

complexity = 0 

for code in codes:

    #Numpad
    numpaths = []
    position = (3,2)
    for digit in code:
        target = numpad[digit]
        numpaths.append(getPath(position, target, numpad_coords))
        position = target
    combinations = [''.join(x) for x in product(*numpaths)]
    #print(combinations)


    #First Controller
    dircombinations = []
    for instruction in combinations:
        paths = []
        position = (0,2)
        for key in instruction:
            target = keypad[key]
            paths.append(getPath(position, target, keypad_coords))
            position = target
        dircombinations += [''.join(x) for x in product(*paths)]
    print(dircombinations)


    #Second Controller
    sdircombinations = []
    for instruction in dircombinations:
    
        paths = []
        position = (0,2)
        for key in instruction:
            target = keypad[key]
            paths.append(getPath(position, target, keypad_coords))
            position = target
        sdircombinations += [''.join(x) for x in product(*paths)]
    #print(len(sdircombinations))

    min = 10*10
    for i in range (0,len(sdircombinations)):
        if len(sdircombinations[i])<min:
            min = len(sdircombinations[i]) 
    print(min)
    complexity += (min*int(code[:3]))

print(complexity)

In [54]:
def solve(combinations):
    minimumcomplexity = 1000000000**10
    #For each potential route
    for option in combinations:
        complexity = 0    

        #For each snippet in the potential route 
        for snippet in option:
            
            #Solve the complexity of this individual snippet
            complexity += snippetsolve((snippet,0))

        if complexity < minimumcomplexity : 
            minimumcomplexity = complexity
    
    return minimumcomplexity

@lru_cache(maxsize=None)
def snippetsolve(st):
    snippet,depth=st 
    paths = []
    snippetdone = []

    position = (0,2)
    
    #Find the path for the next controller
    for key in snippet :
        target = keypad[key]
        paths.append(getPathCache(position,target))
        position = target
    
    
    #Get all options for this new path
    snippetdone += [list(x) for x in product(*paths)]
    

    if depth+1 == 25 :
        mini = 100000000*10
        for option in snippetdone:
            optlength = 0 
            for snippet in option:
                optlength += len(snippet)
            if optlength < mini :
                mini = optlength

        return mini 


    #For each option of this new result
    mini = 10000000000**10

    for option in snippetdone:
        optlength = 0 
        #For each snippet of this new result
        for snippet in option:
            #Solve again deeper and add the length 
            optlength += snippetsolve((snippet,depth+1))
        
        if optlength < mini :
            mini = optlength

    return mini 




In [ ]:
totalcomplexity = 0 

for code in codes:

    #Numpad
    numpaths = []
    position = (3,2)

    #for each digit in the code
    for digit in code:
        target = numpad[digit]
        numpaths.append(getPath(position, target, numpad_coords))
        position = target
    combinations =[]
    combinations += [list(x) for x in product(*numpaths)]
    #print(combinations)

    #All Controllers
    complexity = solve(combinations)
    print(complexity)

    totalcomplexity += (complexity*int(code[:3]))

print(totalcomplexity)
